In [2]:
import numpy as np
import cv2
from ex3_utils import *
from ex2_utils import *
import matplotlib.pyplot as plt

In [ ]:
src = "vids/basketball/00000001.jpg"
image = cv2.imread(src)
region = [188.62,321.46,194.43,210.61,231.38,212.54,225.57,323.39]

image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
if len(region) == 8:
    x_ = np.array(region[::2])
    y_ = np.array(region[1::2])
    region = [np.min(x_), np.min(y_), np.max(x_) - np.min(x_) + 1, np.max(y_) - np.min(y_) + 1]

window = max(region[2], region[3])

left = max(region[0], 0)
top = max(region[1], 0)

right = min(region[0] + region[2], image.shape[1] - 1)
bottom = min(region[1] + region[3], image.shape[0] - 1)
    
position = (region[0] + region[2] / 2, region[1] + region[3] / 2)
size = (int(region[2] * 2), int(region[3] * 2))

width = size[0]
height = size[1]

#Size is width x height as required for both util functions

print(size)

template, mask = get_patch(image, position, size)
print(template.shape)
G = create_gauss_peak(size, 3)
hanning_window = create_cosine_window(size)

P = (template * hanning_window)

P_hat = np.fft.fft2(P)
P_hat_conjugate = np.conjugate(P_hat)
G_hat = np.fft.fft2(G)

H_hat_conjugate = (G_hat * P_hat_conjugate) / ((P_hat * P_hat_conjugate)+1e-5)

H = np.fft.fft2(H_hat_conjugate).real

R = np.fft.ifft2(H_hat_conjugate * P_hat).real
print(f"P_hat magnitude min: {np.abs(P_hat).min()}, max: {np.abs(P_hat).max()}")
print(f"G_hat magnitude min: {np.abs(G_hat).min()}, max: {np.abs(G_hat).max()}")
cv2.imshow("R", R)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(np.max(R, axis=1))

(87, 227)
(227, 87)
P_hat magnitude min: 2.690217414715446, max: 2287952.0
G_hat magnitude min: 0.0, max: 56.54866776461628
[1.00000000e+00 9.45959469e-01 8.00737403e-01 6.06530660e-01
 4.11112291e-01 2.49352209e-01 1.35335283e-01 6.57285286e-02
 2.85655008e-02 1.11089965e-02 3.86592014e-03 1.20385999e-03
 3.35462628e-04 8.36483470e-05 1.86644690e-05 3.72665319e-06
 6.65836262e-07 1.06453888e-07 1.52301641e-08 1.94991717e-09
 2.23443749e-10 2.28932691e-11 2.01544012e-12 2.75831777e-13
 3.05331582e-13 3.21568286e-13 3.14425128e-13 2.75493626e-13
 2.79749714e-13 2.67572654e-13 2.58179973e-13 2.67762153e-13
 2.69298700e-13 2.51354270e-13 2.60695908e-13 2.80225859e-13
 2.86512000e-13 2.68960056e-13 2.38802043e-13 2.37637304e-13
 2.48050035e-13 2.78002306e-13 2.99043265e-13 2.92842458e-13
 2.79949710e-13 2.84770916e-13 2.58364359e-13 2.52889335e-13
 2.91637316e-13 2.79654758e-13 2.27615328e-13 2.53484347e-13
 2.55573978e-13 2.69327625e-13 2.86887944e-13 2.73685178e-13
 2.60460108e-13 2.3642

In [ ]:
src = "vids/basketball/00000009.jpg"
image = cv2.imread(src)
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

template, mask = get_patch(image, position, size)

P = (template * hanning_window)

P_hat = np.fft.fft2(P)
R = np.fft.ifft2(H_hat_conjugate * P_hat).real

x_old = width // 2
y_old = height // 2


y, x = np.unravel_index(R.argmax(), R.shape)
print(y, x)

if x > width / 2:
    x = x - width

if y > height / 2:
    y = y - height

x_new = x_old + x
y_new = y_old + y

print(x_new, y_new)

cv2.imshow("R", R)
cv2.waitKey(0)
cv2.destroyAllWindows()

223 78
34 109
